In [ ]:
from ibapi.client import EClient
from ibapi.wrapper import EWrapper
from ibapi.contract import Contract
from datetime import datetime
from math import isnan

import threading
import time
import pandas
from pandas import DataFrame

In [ ]:
class IBapi(EWrapper, EClient):
    def __init__(self):
        EClient.__init__(self, self)
        self.data = []  # Initialize variable to store candle

    def historicalData(self, reqId, bar):
        print(f'reqId: {reqId} Time: {bar.date} Close: {bar.close}')
        self.data.append([reqId, bar.date, bar.close])

In [ ]:
app = IBapi()
app.connect('127.0.0.1', 7497, 10645)


def run_loop():
    app.run()

In [ ]:
reqId_serial = 1

In [ ]:
# Start the socket in a thread
api_thread = threading.Thread(target=run_loop, daemon=True)
api_thread.start()
time.sleep(1)  # Sleep interval to allow time for connection to server

In [ ]:
def create_contract(curr1, curr2):
    # Create contract object
    contract = Contract()
    contract.symbol = curr1
    contract.secType = 'CASH'
    contract.exchange = 'IDEALPRO'
    contract.currency = curr2
    return contract

In [ ]:
EUR_USD_contract = create_contract('EUR', 'USD')

In [ ]:
# Request historical candles
app.reqHistoricalData(reqId=1, contract=EUR_USD_contract, endDateTime='', durationStr='1 D', barSizeSetting='1 day',
                      whatToShow='MIDPOINT', useRTH=0, formatDate=1, keepUpToDate=False, chartOptions=[])
time.sleep(2)  # sleep to allow enough time for data to be returned

In [ ]:
df = pandas.DataFrame(app.data, columns=['reqId', 'DateTime', 'Close'])

In [ ]:
df['DateTime'] = [datetime.strptime(dt, '%Y%m%d') for dt in df['DateTime']]

In [ ]:
df

In [ ]:
def request_data(contract):
    global reqId_serial
    reqId_serial += 1
    app.reqHistoricalData(reqId=reqId_serial, contract=contract, endDateTime='', durationStr='1 D',
                          barSizeSetting='1 day',
                          whatToShow='MIDPOINT', useRTH=0, formatDate=1, keepUpToDate=False, chartOptions=[])
    time.sleep(0.5)
    if app.data[len(app.data) - 1][0] != reqId_serial:
        return float('NaN')
    else:
        return app.data[len(app.data) - 1][2]

In [ ]:
print(request_data(EUR_USD_contract))

In [ ]:
def fetch_exc_rate(base, quote):
    contract = create_contract(base, quote)
    return request_data(contract)

In [ ]:
currencies = ['USD', 'EUR', 'GBP', 'JPY', 'AUD']

In [ ]:
def fetch_all(curr_list):
    matrix = DataFrame(columns=curr_list, index=curr_list)
    for i in range(0, len(curr_list)):
        for j in range(0, len(curr_list)):
            if i == j:
                matrix[curr_list[i]][curr_list[j]] = 1
            else:
                matrix[curr_list[i]][curr_list[j]] = fetch_exc_rate(curr_list[i], curr_list[j])
    return matrix

In [ ]:
exchange_rates = fetch_all(currencies)

In [ ]:
def fill_in_nan(matrix):
    for col in matrix.columns:
        for row in matrix.index:
            if isnan(matrix[col][row]) & (not isnan(matrix[row][col])):
                matrix[col][row] = 1 / matrix[row][col]
    return matrix

In [ ]:
def check_all_data(matrix):
    return not matrix.isnull().values.any()


In [ ]:
exchange_rates = fill_in_nan(exchange_rates)

In [ ]:
check_all_data(exchange_rates)

In [ ]:
exchange_rates = fetch_all(currencies)

In [ ]:
def fill_in_nan(matrix):
    for col in matrix.columns:
        for row in matrix.index:
            if isnan(matrix[col][row]) & (not isnan(matrix[row][col])):
                matrix[col][row] = 1 / matrix[row][col]
    return matrix

In [ ]:
def check_all_data(matrix):
    return not matrix.isnull().values.any()


In [ ]:
exchange_rates = fill_in_nan(exchange_rates)

In [ ]:
check_all_data(exchange_rates)

In [ ]:
exchange_rates

In [ ]:
def check_arbitrage(result_dict, matrix, curr1, curr2, curr3):
    arbitrage_amount = matrix[curr1][curr2] * matrix[curr2][curr3] * matrix[curr3][curr1] - 1
    result_dict[curr1 + '.' + curr2 + '.' + curr3] = arbitrage_amount
    arbitrage_amount_rev = matrix[curr1][curr3] * matrix[curr3][curr2] * matrix[curr2][curr1] - 1
    result_dict[curr1 + '.' + curr3 + '.' + curr2] = arbitrage_amount_rev
    print(curr1 + '->' + curr2 + '->' + curr3 + '->' + curr1 + ': ' + "{:.6f}".format(arbitrage_amount))
    print(curr1 + '->' + curr3 + '->' + curr2 + '->' + curr1 + ': ' + "{:.6f}".format(arbitrage_amount_rev))
    return arbitrage_amount

In [ ]:
def check_all_arbitrage(result_dict, matrix, currency_list):
    for i in range(0, len(currency_list)):
        for j in range(i + 1, len(currency_list)):
            for k in range(j + 1, len(currency_list)):
                check_arbitrage(result_dict, matrix, currency_list[i], currency_list[j], currency_list[k])
    return result_dict

In [ ]:
results = dict()
results = check_all_arbitrage(results, exchange_rates, currencies)

In [ ]:
df_results = DataFrame.from_dict(results,orient='index').reset_index().rename(columns={'index':'Paths','0':'Profits'})

In [ ]:
df_results

In [ ]:
results

In [ ]:
df_results.reset_index()

In [ ]:
df_results.reset_index().rename(columns={'index':'Paths','0':'Profits'})

In [ ]:
results

In [ ]:
df_results.reset_index()

In [ ]:
df_results.reset_index().rename(columns={'index':'Paths','0':'Profits'})

In [39]:
df_results.reset_index()

TypeError: unhashable type: 'dict'

In [41]:
df_results.reset_index().rename(columns={'index':'Paths','0':'Profits'})

,Paths,0
0,USD.EUR.GBP,-0.000033
1,USD.GBP.EUR,0.000033
2,USD.EUR.JPY,-0.000003
3,USD.JPY.EUR,0.000003
4,USD.EUR.AUD,0.000010
5,USD.AUD.EUR,-0.000010
6,USD.GBP.JPY,0.000017
7,USD.JPY.GBP,-0.000017
8,USD.GBP.AUD,0.000013
9,USD.AUD.GBP,-0.000013
